<a href="https://colab.research.google.com/github/romulo-souza/IA/blob/main/Aprendizado_De_Maquina_Supervisionado/DecisionTree_Modularizado_BreastCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base de Dados: Breast Cancer
* Base de dados: https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data (baixar e colocar em arquivos no Colab)
* Classe: Diagnosis (M = malignant, B = benign)
* Usaremos o algoritmo de arvore de decisão (decision tree) do scikitlearn. LINK: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [ ]:
import numpy as np
import pandas as pd

# Modelo machine learning
from sklearn.model_selection import train_test_split #amostragem por holdout
from sklearn.preprocessing import StandardScaler #normalização, nesse caso será z score
from sklearn.tree import DecisionTreeClassifier #Algortimo de classificação é a arvore de decisão
from sklearn.preprocessing import LabelEncoder #transformar dados categoricos (classe diagnosis) em numéricos

# Amostragem por Validação Cruzada
from sklearn.model_selection import (
    KFold, #modelo de validação cruzada
    LeaveOneOut, #modelo de validação cruzada
    StratifiedKFold, #modelo de validação cruzada
    cross_validate #classe, função que realiza a validação cruzada
)

# Métricas
from sklearn.metrics import (recall_score,
                             accuracy_score,
                             precision_score,
                             f1_score)
from sklearn.metrics import classification_report # Extrato geral das matricas de classificação

In [ ]:
#Função para carregar/gerar Base de Dados (Dataframe)
def carregaBaseDados(nome):
  return pd.read_csv(nome)#retorna um dataframe


# Pré-processamento
* Parametros da função
* dataframe - > dataframe que foi retornado na função carregaBaseDados
* rem_cols - > Colunas a serem removidas, serão passadas em forma de uma lista
* class_column -> Nome da coluna que é a classe alvo, nesse caso é a coluna diagnosis
* normalization_cols -> Especificar quais colunas serão normalizadas,  serão passadas em forma de uma lista

obs.: A normalização deve ser aplicada apenas às colunas que são originalmente numéricas na base de dados. As colunas categóricas, mesmo após serem transformadas em números inteiros pelo LabelEncoder, não devem ser normalizadas. Isso ocorre porque o LabelEncoder atribui números inteiros arbitrários às categorias, mas esses números não têm uma ordem ou escala significativa. Normalizar esses dados pode introduzir distorções que não fazem sentido para o modelo.




In [ ]:
def preProcessamento(dataframe, rem_cols, class_column, normalization_cols):

  #Remoção das colunas irrelevantes
  dataframe.drop(rem_cols, axis = 1, inplace = True) #axis = 1 pois é uma coluna, inplace = True para substituir o dataframe original por esse

  #Transformar dados categóricos da classe 'diagnosis' em dados numéricos
  le = LabelEncoder()
  dataframe[class_column] = le.fit_transform(dataframe[class_column]) #os valores categoricos da coluna que tem a classe 'diagnosis' serão substiuídos por uma coluna com codificação de rótulos categóricos
  #Normalização dos dados
  scaler = StandardScaler()
  dataframe[normalization_cols] = scaler.fit_transform(dataframe[normalization_cols]) #Faz a normalização das colunas selecionadas e já coloca no dataframe

  return dataframe #retorna o dataframe após as modificaçoes do pre-processamento

In [ ]:
df = carregaBaseDados("data.csv")


In [ ]:
df.info()# verificar a nossa base de dados para o pre-processamento, percebe-se que todas colunas estao com 569 amostras com exceção da coluna 32 que nao possui nada(gerado por ruído/lixo), ou seja precisará ser removida. Outra coluna que pode ser removida é o id

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [ ]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [ ]:
df = preProcessamento(df, ['id','Unnamed: 32'], 'diagnosis', ['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'])

In [ ]:
df.info() #verificar como ficou após o pre-processamento

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    int64  
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [ ]:
#Separar os atributos (previsores) da classe (diagnosis) (X -> previsores, y -> classe)
def separaClasse(dataframe, classe):
  X = dataframe.drop(classe, axis = 1)
  y = dataframe[classe]
  return X,y

# Amostragem Holdout

In [ ]:
#Separa os conjuntos em treino e teste (70%/30%)
def separaTreinoTeste(X, y):
  return train_test_split(X,y,test_size=0.3)

#Amostragem por validação cruzada, no caso 'K-fold Cross-validation' para testar

Parametros do cross_validate():
* O model a ser passado é uma string, porém, para ele interpretar como uma função de modelo de algoritmo que será executado, precisamos usar o eval(), que converte essa string em um objeto da classe correspondente e permite que o modelo seja instanciado;
* X,y sao os previsores e a classe respectivamente;
* Scoring é a métrica que será utilizada para verificar o desempenho do classificador;
* cv é o tipo de validação cruzada que estamos usando.

In [ ]:
def KFCross(model, X, y):
  kf = KFold(n_splits = 10, shuffle = True) #n_splits -> numeros de partições (folds). shuffle -> para cada partição faz um novo embaralhamento para garantir que os dados estejam independentes entre si
  #objeto que fará a validação cruzada
  clf = cross_validate(
      eval(model),
      X,y,
      scoring = 'accuracy',
      cv = kf
  )
  return clf


#Amostragem por validação cruzada, no caso 'Stratified K-fold Cross-validation' para testar

In [ ]:
def Skf(model, X, y):
  skf = StratifiedKFold(n_splits = 10, shuffle = True)
  #objeto que fará a validação cruzada
  clf = cross_validate(
      eval(model),
      X,y,
      scoring = 'accuracy',
      cv = skf
  )
  return clf

#Modelo preditivo

In [ ]:
#Gera o modelo preditivo
def geraModelo(modelo, X_train, y_train):
  modelo = eval(modelo) #modelo do parametro será passado por uma string, entao precisamos fazer seu eval para ter efeito no codigo
  modelo.fit(X_train, y_train) #treina o modelo
  return modelo


#Métricas

In [ ]:
def metricaReport(y_test, y_pred): #y_test ->gabarito do conjunto teste, y_pred -> o que foi predito pelo classificador(algoritmo)
  print(classification_report(y_test, y_pred))#imprimir no formato da função classification report

#Teste com os dados

In [ ]:
#Separar atributos previsores e classe
X, y = separaClasse(df, 'diagnosis')


# Testando com método Holdout

In [ ]:
#Gerar conjunto treino e teste
X_train, X_test, y_train, y_test = separaTreinoTeste(X, y) #método Holdout

In [ ]:
#Avalia os dados (nesse caso o modelo é de arvore de decisão)
#Aqui estamos utilizando o método de amostragem Holdout
modelo = geraModelo('DecisionTreeClassifier()', X_train, y_train)
score = modelo.score(X_test, y_test)#retorna automaticamente a acurácia do modelo
y_pred = modelo.predict(X_test) #retorna as saídas preditas pelo classificador
print(score)

0.9005847953216374


In [ ]:
#Avalia o modelo com mais métricas e detalhes
metricaReport(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.98      0.86      0.92       109
           1       0.80      0.97      0.88        62

    accuracy                           0.90       171
   macro avg       0.89      0.92      0.90       171
weighted avg       0.91      0.90      0.90       171



#Testando com métodos de validação cruzada
* A função 'cross_validate', dentro das funções criadas 'KFCross' e 'Skf', retorna um dicionário, e dentro desse dicionário estamos mostrando somente os scores do conjunto teste(chave 'test_score') e mostra também a média dos scores dos testes, pois são 10 testes nesse caso.
* De forma geral, é mais comum se apoiar por abordagens de validação cruzada do que pelo método Holdout para se fazer as análises

In [ ]:
#KF
cv = KFCross('DecisionTreeClassifier()', X,y)
print(f"{cv['test_score']}\nMedia: {np.mean(cv['test_score'])}")

[0.96491228 0.85964912 0.89473684 0.96491228 0.94736842 0.94736842
 0.89473684 0.87719298 0.89473684 0.92857143]
Media: 0.9174185463659148


In [ ]:
#KF estratificado
cv = Skf('DecisionTreeClassifier()', X,y)
print(f"{cv['test_score']}\nMedia: {np.mean(cv['test_score'])}")

[0.89473684 0.84210526 0.96491228 0.96491228 0.96491228 0.92982456
 0.94736842 0.9122807  0.94736842 0.92857143]
Media: 0.9296992481203008
